In [ ]:
import tensorflow as tf
from GAN import GAN
import numpy as np
import os, gc
import matplotlib.pyplot as plt
import mnist
from skimage.transform import resize
%matplotlib inline

# Experiments with WGAN and mnist results
I tried different options with watered down but result images weren't good</br>
<ol>
    <li> WGAN GP takes a long time to train
    <li> WGAN does not create good images at short period of times
   
</ol>


In [ ]:
def transform_image(img, code):
    if code == 0:
        return(img)
    
    if code == 1:
        ang = np.random.randint(low=-10, high=10)
        p = np.int(np.ceil(np.sin(np.deg2rad(np.abs(ang)))*img.shape[0]))
        img = rotate(img, ang)
        img[:, 0:p] = -1
        img[:,-p::] = -1
        img[0:p, :] = -1
        img[-p::, :] = -1
        
        return(img)
    
    
    if code == 2:
        sft = np.random.randint(-4, 4)
        tr = SimilarityTransform(scale=1, rotation=0, translation=[sft,sft])
        img = warp(img, tr)
        if sft>0:
            img[:,-sft::] = -1
            img[-sft::,:] = -1
        else:
            img[:,0:-sft] = -1
            img[0:-sft,:] = -1
        return(img)
        

In [ ]:
#No TRnsform
mn = mnist.Mnist(path=".")
imgs, lbls, _ = mn.get_batch(60000)
imgs = (imgs - 0.5)*2


def get_batch(batch_size):
    global imgs
    rnd_idxs = np.random.choice(np.arange(imgs.shape[0]), batch_size)
    #for idx in rnd_idxs:
        #code = np.random.choice(np.arange(3), 1)[0]
        #imgs[idx] = transform_image(imgs[idx], code=code)
    return (imgs[rnd_idxs])    
    
        

# DCGAN different settig mnist

In [ ]:
ns_op=[0, 1, 2, 3]
mb_op =[[4, 5], [2, 3]]

def get_pars():
    return(ns_op[np.random.randint(0, len(ns_op))], np.random.randint(100, 500), np.round(np.random.uniform(0.8, 0.92), 2), mb_op[np.random.randint(0, len(mb_op))])

In [ ]:
cases = list()

In [ ]:
def try_settings():
    z_dim = 100
    tf.reset_default_graph()
    gan = GAN(verbose=True)
    layers = list()
    nsl, dec, sm, mba = get_pars()
    cases.append((nsl, dec, sm, mba))
    ns=np.zeros(3, int).tolist()
    if nsl>0:
        for i in range(nsl):
            ns[i] = tf.train.exponential_decay(0.1, tf.Variable(0, trainable=False), dec, 0.95, staircase=True)

    G_layers = [[1024, [4, 4], (1, 1), "valid", tf.random_normal_initializer(stddev=0.02), 1, "r", None, 0],
                [512, [4, 4], (2, 2), "same", tf.random_normal_initializer(stddev=0.02), 1, "r", None, 0],
                [256, [4, 4], (2, 2), "same", tf.random_normal_initializer(stddev=0.02), 1, "r", None, 0],
                [128, [4, 4], (2, 2), "same", tf.random_normal_initializer(stddev=0.02), 1, "r", None, 0],
                [1, [4, 4], (2, 2), "same", tf.random_normal_initializer(stddev=0.02), 0, "th", None, 0]]

    D_layers = [[128, [4, 4], (2, 2), 'same', tf.random_normal_initializer(stddev=0.02), 0, 'lr', None,  ns[0]],
                [256, [4, 4], (2, 2), 'same', tf.random_normal_initializer(stddev=0.02), 1, 'lr', None, ns[1]],
                [512, [4, 4], (2, 2), 'same', tf.random_normal_initializer(stddev=0.02), 1, 'lr', None, ns[2]],
                [1024, [4, 4], (2, 2), 'same', tf.random_normal_initializer(stddev=0.02), 1, 'lr', mba, 0],
                [1, [4, 4], (1, 1), 'valid', tf.random_normal_initializer(stddev=0.02), 0, 'l', None, 0]]

    x = tf.placeholder(tf.float32, [None, 28, 28])
    xr = tf.image.resize_images(tf.expand_dims(x, axis=3), (64, 64))

    gargs = gan.build_graph(G_layers, D_layers, xr,
                           tf.placeholder(tf.float32, [None, 1, 1, z_dim]), True, sm, ["RMS"], 2e-4, True)

    gargs = [g for g in gargs]
    gargs[1] = x
    gargs = tuple(gargs)

    gan.train(60,  3000, 10, get_batch, "n", z_dim, True, "summary\\GAN_mnist_case_dn_{}".format(len(cases)), "Model\\GAN_mnist_case_dn_{}".format(len(cases)), None, None, gargs)

In [ ]:
for _ in range(8):
    try_settings()

In [ ]:
np.savez("GAN_cases_dn.npz", [str(c) for c in cases])

# Compare the results

In [ ]:
np.load("GAN_cases_dn.npz")["arr_0"]

In [ ]:
def generate_img(save_path):
    z_dim=100
    G_layers = [[1024, [4, 4], (1, 1), "valid", tf.random_normal_initializer(stddev=0.02), 1, "r", None, 0],
                [512, [4, 4], (2, 2), "same", tf.random_normal_initializer(stddev=0.02), 1, "r", None, 0],
                [256, [4, 4], (2, 2), "same", tf.random_normal_initializer(stddev=0.02), 1, "r", None, 0],
                [128, [4, 4], (2, 2), "same", tf.random_normal_initializer(stddev=0.02), 1, "r", None, 0],
                [1, [4, 4], (2, 2), "same", tf.random_normal_initializer(stddev=0.02), 0, "th", None, 0]]

    tf.reset_default_graph()
    gan = GAN(verbose=False)
    z = tf.placeholder(shape=[20, 1, 1, z_dim], dtype=tf.float32)
    graph, sess, z, G_col, G_out = gan.build_inference_graph(G_layers, z, save_path)
    z_in = gan._get_z_code(20, "n", z_dim)
    images = sess.run(G_out, feed_dict={z:z_in})
    sess.close()
    return (images)


In [ ]:
img_list = []
for i in range(1,9):
    img = np.clip(generate_img("Model\\GAN_mnist_case_dn_{}-2999".format(i)).squeeze()/2+0.5, 0, 1)
    img_list.append(img)

fig, ax = plt.subplots(4, 2)
fig.set_size_inches(15, 25)
for i, img in enumerate(img_list):
    ax[i//2,i-2*(i//2)].set_title(i)
    ax[i//2,i-2*(i//2)].imshow(np.vstack([np.hstack(img[0:5,...]), np.hstack(img[5:10,...]),
                     np.hstack(img[10:15,...]), np.hstack(img[15:20,...])]), cmap="gray") 
    
    
plt.show()

# Good settings on mnist
<table align="center">
    <tr>
        <th>layers with noise</th>
        <th>decay steps of 3000 epochs</th>
        <th>smoothing</th>
        <th>mini-batch discr.</th>
    </tr>
    <tr><td>3</td><td>465</td><td>0.91</td><td>[2, 3]</td></tr>
    <tr><td>3</td><td>390</td><td>0.85</td><td>[2, 3]</td></tr>
    <tr><td>3</td><td>440</td><td>0.87</td><td>[2, 3]</td></tr>
    <tr><td>3</td><td>489</td><td>0.82</td><td>[2, 3]</td></tr>
   
</table>

<ol>
    <li>so adding noise to all discriminator layers are good option
    <li>also decay of 14% of total epochs are good like 489 is better that 390

</ol></p>

